In [4]:
import pandas as pd
from scipy.stats import norm
import numpy as np
import pandas as pd

import datetime

In [16]:


race_features = pd.read_pickle("data/race_features.pickle")
races = pd.read_pickle("data/races.pickle")

HK odds or fair odds: 2.5 or 2.5-1 is 1/2.5

US odds: HK odds of 2.5 is 1.5-1 US odds

d = probability

p = odds

Expected value = $p \times (10d - 10) + (1-p) \times (-10) $

Expected value = $dp > 1$

public odd = Fair odd x 0.82 = $\frac{1}{BEtting Ratio} \times 0.82$

$Betting Ratio = \frac{1}{Public odd} \times 0.82$

In [39]:
races['public_odds'] = .82 / races.loc[:, "win_odds"]

races['public_variance'] = races['public_odds'] * (1- races['public_odds'])

#race_features[['public_odds', 'public_variance']]
temp = races.groupby("jockey_id")['public_odds'].sum().to_frame(name="expected_wins")
temp['actual_wins'] = races.groupby("jockey_id")['pla'].agg(lambda x: sum([(1 if r == "01" else 0) for r in x]))
temp
norm.cdf(1.0, 1.7497835, 1.3608731)

#race_features.columns

0.29083133930970545



## Adjusted place percentage
1.2 * #win + 1.1 * #2nd + 1 * #3rd / #races * 1/3 


In [ ]:
race_features.loc[1, ""]

### Ainslie's conversion method
To convert finish time to horse length, one sec is about 6 horse lengths
To convert weight carried to horse length: 4lb = 1 horse length in sprints (<1200m), 3lb = 1 horse lengths in 1600m, 2lb = 1 horse length in 1800m, 1lb = 1 horse length in 2000m. 



How long is long_rest?
Finishing_time is biased cause many jockey will give up if they know they can't win. 
Multinomial Logistica Regression needs independent features. 

### Estimating Placed Probability by Winning Probability

#### Harville Formula

$P(\text{horse No. 1 wins and horse no.2 runs second}) = \frac{P(x_{1}) \dot P(x_{2})}{1 - P(x_{1})}$ 

This formula is wrong since it does not consider dependence.

#### Normal Formula

Horse number k wins and horse number 1 runs second:

$\int_{-\infty}^{\infty}(1 - F_{k}(x)) \dot f_{1}(x) \prod_{i\neq k, i \neq l} F_{i}(x)dx$

#### Benter Correction

$ P(m, n) = P_{1}(m) \frac{P_{2}(n)}{1-P_{2}(m)}$

Where $P_{1}(x)$ is the winning probability of horse number x. The $P_{2}(x)$ is calculated by : 

$P_{2}(x) = $ 
$\frac{[P_{1}(x)]^{k_{2}}}{\sum_{j=1}^{N}[P_{1}(j)]^{k_{2}}} $

Then we find $k_{2}$ by MLE. $k_{2} = 0.81$ For example, # 1 = 0.4, # 2 = 0.3, # 3 = 0.2, # 4 = 0.1 Then $P_{2}(1) = 0.372044$ , $P_{2}(2) = 0.294709$, $P_{2}(3) = 0.212207$, $P_{2}(4) = 0.121039$

Thus, $P(1,2) = P_{1}(1)\times\frac{P_{2}(2)}{1 - P_{2}(1)} = 0.4 \times \frac{0.294709}{1 - 0.372044} = 0.187726$

probability of first, second, and third be $P(m, n, q)$:

$P(m, n, q) = P_{1}(M) \times \frac{P_{2}(n)}{1 - P_{2}(m)} \times \frac{P_{3}(q)}{1 - P_{3}(m) - P_{3}(n)}$

Where this time the $P_{3}(x) = $

$\frac{[P_{1}(x)]^{k_{3}}}{\sum_{j=1}^{N}[P_{1}(j)]^{k_{3}}}$

And $k_{3} = 0.65$


### Kelly Criterion

#### Two Horse example

$X_{n} = X_{0}(1 + f)^{A}(1-f)^{B}$
Where A is times horse win, and B is the number of times lost. Such that A + B = n. $f$ is the proportion of capital we should bet. Then the result $X_{n} is the remaining capital after betting k times. 

We write $\frac{X_{n}}{X_{0}}$, the increment of capital as an exponential growth rate. 

$\frac{X_{n}}{X_{0}} = \text{e}^{n \text{log}(\frac{X_{n}}{X_{0}})^{\frac{1}{n}}}$

$G_{n}(f)$ is the exponential rate of increase of capital after each bets. 

$G_{n}(f) = log(\frac{X_{n}}{X_{0}})^{\frac{1}{n}} = log(\frac{X_{0}(1 + f)^{A}(1 - f)^{B}}{X_{0}})^{\frac{1}{n}} = log((1+f)^{\frac{A}{n}}(1 - f)^{\frac{B}{n}}) = \frac{A}{n}log(1 + f) + \frac{B}{n}log(1 - f)$

Let $g(f) = E[G_{n}(f)]$, The expectaion of $G_{n}(f)$

$g(f) = E(\frac{A}{n}log(1 + f) + \frac{B}{n}log(1 - f)) = p log(1 + f) + q log(1 - f) = p log(1 + f) + (1-p) log(1-f)$

We maximize $g(f)$ for f to find the proportion of $f$ investment that maximizes $X_{n}$ growth rate, the remaining capital exponential growth rate. We find $g'(f)$

$\frac{d}{df}[p log(1 + f) + q log(1 - f)] = \frac{p}{1 + f} - \frac{q}{1 - f} = \frac{p - q - f}{(1+f)(1-f)}$

Set $g'(f) = 0$

$f = p - q = 2p - 1$


In [ ]:
from sympy import * 

In [10]:

x, b, p = symbols('x b p')

In [11]:
y = p * log(1 + b*x) + (1-p) * log(1-x)
solve(diff(y, x), x)



[(b*p + p - 1)/b]

In [20]:
p, q, f = symbols("p q f")
y = (p * log(1 + f)) + (q * log(1-f))
df = diff(y, f)
display(df)
display(solve(df, f))


p/(f + 1) - q/(-f + 1)

[(p - q)/(p + q)]

#### Generalized

$p$ is the winning probability
$q$ is the other's horse winning probability
where $p > q$

g(f) is the expectation of expoential growth rate. 

$g(f) = \sum_{k=1}^{m} P_{k} log(1 + f_{k}o_{k} - f)$

where $P_{k}$ is the winning probability of horse # k, $f_{k}$ is the proportion of the tital capital you will bet on horse # k. $f = \sum_{i=1}^{m}f_{i}$ is the proportion of capital that you have bet in this race. $o_{k}$ is the actual win odd of horse # k.

$g'(f) = \frac{\partial}{\partial f} \sum_{k=1}^{m} P_{k} log(1 + f_{k}o_{k} - f) $

Let $b$ be the real odds

$\frac{d}{dx}\sum p_{i} log(1 + b_{i} x) = 0$

$\sum \frac{p_{i}b_{i}}{1 + b_{i}{x}} = 0$


In [58]:
p1, p2, p3, f1, f2, f3, o1, o2, o3, f = symbols("p1 p2 p3 f1 f2 f3 o1 o2 o3 f")

#f = sum(f1, f2, f3)
y = (p1 * log(1 + f1 * o1 - (f1 + f2 + f3))) + (p2 * log(1 + f2 * o2 - (f1 + f2 + f3))) + (p3 * log(1 + f3 * o3 - (f1 + f2 + f3)))
y1 = (p1 * log(1 + f1 * o1 - (f1 + f2)))

#df = diff(y, f1)
#display(df)
#df_solve = solve(df, f1)
#df_simp = simplify(df_solve)
#display(df_simp)

#p_k, f_k, o_k = symbols("p_k f_k o_k")
#f = Function("f")

#diff((p1 * log(1 + (f1 * o1) - 0.3)) , f1)

y = ((p1 * log(1 + (f1 * o1) - (f1 + f2 + f3))) + (p2 * log(1 + (f2 * o2) - (f1 + f2 + f3))) + (p3 * log(1 + (f3 * o3) - (f1 + f2 + f3))))
#solve(diff(y, f1), f1)

p1, p2, p3, b1, b2, b3, x = symbols("p1 p2 p3 b1 b2 b3 x")
y = (p1 * log(1 + b1 * x)) + (p2 * log(1 + b2 * x)) + (p3 * log(1 + b3 * x))
k = simplify(solve(diff(y, x), x))
k

[(-b1*b2*p1 - b1*b2*p2 - b1*b3*p1 - b1*b3*p3 - b2*b3*p2 - b2*b3*p3 + sqrt(b1**2*b2**2*p1**2 + 2*b1**2*b2**2*p1*p2 + b1**2*b2**2*p2**2 - 2*b1**2*b2*b3*p1**2 - 2*b1**2*b2*b3*p1*p2 - 2*b1**2*b2*b3*p1*p3 + 2*b1**2*b2*b3*p2*p3 + b1**2*b3**2*p1**2 + 2*b1**2*b3**2*p1*p3 + b1**2*b3**2*p3**2 - 2*b1*b2**2*b3*p1*p2 + 2*b1*b2**2*b3*p1*p3 - 2*b1*b2**2*b3*p2**2 - 2*b1*b2**2*b3*p2*p3 + 2*b1*b2*b3**2*p1*p2 - 2*b1*b2*b3**2*p1*p3 - 2*b1*b2*b3**2*p2*p3 - 2*b1*b2*b3**2*p3**2 + b2**2*b3**2*p2**2 + 2*b2**2*b3**2*p2*p3 + b2**2*b3**2*p3**2))/(2*b1*b2*b3*(p1 + p2 + p3)),
 -(b1*b2*p1 + b1*b2*p2 + b1*b3*p1 + b1*b3*p3 + b2*b3*p2 + b2*b3*p3 + sqrt(b1**2*b2**2*p1**2 + 2*b1**2*b2**2*p1*p2 + b1**2*b2**2*p2**2 - 2*b1**2*b2*b3*p1**2 - 2*b1**2*b2*b3*p1*p2 - 2*b1**2*b2*b3*p1*p3 + 2*b1**2*b2*b3*p2*p3 + b1**2*b3**2*p1**2 + 2*b1**2*b3**2*p1*p3 + b1**2*b3**2*p3**2 - 2*b1*b2**2*b3*p1*p2 + 2*b1*b2**2*b3*p1*p3 - 2*b1*b2**2*b3*p2**2 - 2*b1*b2**2*b3*p2*p3 + 2*b1*b2*b3**2*p1*p2 - 2*b1*b2*b3**2*p1*p3 - 2*b1*b2*b3**2*p2*p3 - 2*b1*b2

In [67]:
k[0].subs(p1, 0.7).subs(p2, 0.2).subs(p3, 0.1).subs(b1, -1).subs(b2, 10).subs(b3, 30)

-0.0578343329665600

In [94]:
adj_prob = [0.043578, 0.174312, 0.008716, 0.087156, 0.054472, 0.108945, 0.008716, 0.043578, 0.108945, 0.102536, 0.058104, 0.108945, 0.07263, 0.019368]
your_line = [20, 5, 100, 10, 16, 8, 100, 20, 8, 8.5, 15, 8, 12, 45]
#adj_odd = [22.9, 5.7, 114.7, 11.5, 18.4, 9.2, 114.7, 22.9, 9.2, 9.8, 17.2, 9.2, 13.8, 51.6]
real_odd = [16, 7.2, 61, 12, 6.6, 5.8, 68, 15, 7.8, 8.5, 61, 5.4, 74, 18]
real_prob = [1/o for o in real_odd]


y = (p1 * log(1 + b1 * x)) + (p2 * log(1 + b2 * x)) + (p3 * log(1 + b3 * x))
y = y.subs(p1, 0.7).subs(p2, 0.2).subs(p3, 0.1).subs(b1, -1).subs(b2, 10).subs(b3, 30)
k = simplify(solve(diff(y, x), x))
display(y)
display(k)

y.subs(x, k[1])

def symbolic_kelly(p, b):
    y = 0
    x = symbols("x")
    for (pi, bi) in zip(p, b):
        y += pi * log(1 + bi * x)
    display(y)
    display(diff(y, x))
    x_max = solve(diff(y, x), x)
    display(x_max)
    display([y.subs(x, xx) for xx in x_max])
    return (max(x_max), y.subs(x, max(x_max)), x_max)

symbolic_kelly([0.7, 0.2, 0.1], [-1, 10, 30])

#symbolic_kelly(adj_prob, real_odd)[1].evalf()

0.7*log(-x + 1) + 0.2*log(10*x + 1) + 0.1*log(30*x + 1)

[-0.0578343329665600, 0.247834332966560]

0.7*log(-x + 1) + 0.2*log(10*x + 1) + 0.1*log(30*x + 1)

3.0/(30*x + 1) + 2.0/(10*x + 1) - 0.7/(-x + 1)

[-0.0578343329665600, 0.247834332966560]

[-0.164140557367512 + 0.1*I*pi, 0.263191478105847]

(0.247834332966560,
 0.263191478105847,
 [-0.0578343329665600, 0.247834332966560])

In [70]:
from scipy.optimize import minimize
import numpy as np

#numerical solution
def fofx(x):
    return 0.7 * log(1 + (-1) * x) + 0.2 * log(1 + 10 * x) + 0.1 * log(1 + 30 * x)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
minimize(fofx, method='nelder-mead', options={'xtol': 1e-8, 'disp':True})

TypeError: minimize() missing 1 required positional argument: 'x0'

In [ ]:
k[0].subs

In [ ]:
k[0].evalf

https://www.stat.berkeley.edu/~aldous/157_2016/Slides/lecture_2.pdf?mod=article_inline




From this link
https://www3.cs.stonybrook.edu/~skiena/691/2007/lectures/Kelly.pdf
, suggests $g(f)$'s approximation

$g'(f) \approx \frac{\partial}{\partial f} f \sum P_{i} o_{i} - f^{2}\sum p_{i}o_{i}^{2} / 2 $

$g'(f) = 0$ then solve for $f$

$f = (\sum p_{i}x_{i}) / (\sum p_{i} x_{i}^{2})$

The variance is $\sum p_{i}x_{i}^{2} - (\sum p_{i}x_{i})^{2}$


In [51]:
adj_prob = [0.043578, 0.174312, 0.008716, 0.087156, 0.054472, 0.108945, 0.008716, 0.043578, 0.108945, 0.102536, 0.058104, 0.108945, 0.07263, 0.019368]
your_line = [20, 5, 100, 10, 16, 8, 100, 20, 8, 8.5, 15, 8, 12, 45]
#adj_odd = [22.9, 5.7, 114.7, 11.5, 18.4, 9.2, 114.7, 22.9, 9.2, 9.8, 17.2, 9.2, 13.8, 51.6]
real_odd = [16, 7.2, 61, 12, 6.6, 5.8, 68, 15, 7.8, 8.5, 61, 5.4, 74, 18]
real_prob = [1/o for o in real_odd]

def kelly_fraction(pred, real):
    numerator = sum([f[0]*f[1] for f in zip(pred, real)])
    denominator = sum([f[0]* f[1] * f[1] for f in zip(pred, real)])
    return numerator / denominator

kelly_fraction(adj_prob, real_prob)


7.164455222141723

In [47]:
p1, p2, p3, f1, f2, f3, o1, o2, o3, f = symbols("p1 p2 p3 f1 f2 f3 o1 o2 o3 f")

#f = sum(f1, f2, f3)
y = (p1 * log(1 + f1 * o1 - (f1 + f2 + f3))) + (p2 * log(1 + f2 * o2 - (f1 + f2 + f3))) + (p3 * log(1 + f3 * o3 - (f1 + f2 + f3)))
y1 = (p1 * log(1 + f1 * o1 - (f1 + f2)))

#df = diff(y, f1)
#display(df)
#df_solve = solve(df, f1)
#df_simp = simplify(df_solve)
#display(df_simp)

#p_k, f_k, o_k = symbols("p_k f_k o_k")
#f = Function("f")

#diff((p1 * log(1 + (f1 * o1) - 0.3)) , f1)

y = ((p1 * log(1 + (f1 * o1) - (f1 + f2 + f3))) + (p2 * log(1 + (f2 * o2) - (f1 + f2 + f3))) + (p3 * log(1 + (f3 * o3) - (f1 + f2 + f3))))
solve(diff(y, f1), f1)


[(f2*o1*o2*p1/2 + f2*o1*o2*p3/2 - f2*o1*p1 - f2*o1*p2/2 - f2*o1*p3/2 - f2*o2*p1/2 - f2*o2*p3/2 + f2*p1 + f2*p2 + f2*p3 + f3*o1*o3*p1/2 + f3*o1*o3*p2/2 - f3*o1*p1 - f3*o1*p2/2 - f3*o1*p3/2 - f3*o3*p1/2 - f3*o3*p2/2 + f3*p1 + f3*p2 + f3*p3 + o1*p1 + o1*p2/2 + o1*p3/2 - p1 - p2 - p3 - sqrt(f2**2*o1**2*o2**2*p1**2 + 2*f2**2*o1**2*o2**2*p1*p3 + f2**2*o1**2*o2**2*p3**2 + 2*f2**2*o1**2*o2*p1*p2 - 2*f2**2*o1**2*o2*p1*p3 - 2*f2**2*o1**2*o2*p2*p3 - 2*f2**2*o1**2*o2*p3**2 + f2**2*o1**2*p2**2 + 2*f2**2*o1**2*p2*p3 + f2**2*o1**2*p3**2 - 2*f2**2*o1*o2**2*p1**2 - 4*f2**2*o1*o2**2*p1*p3 - 2*f2**2*o1*o2**2*p3**2 - 2*f2**2*o1*o2*p1*p2 + 2*f2**2*o1*o2*p1*p3 + 2*f2**2*o1*o2*p2*p3 + 2*f2**2*o1*o2*p3**2 + f2**2*o2**2*p1**2 + 2*f2**2*o2**2*p1*p3 + f2**2*o2**2*p3**2 - 2*f2*f3*o1**2*o2*o3*p1**2 - 2*f2*f3*o1**2*o2*o3*p1*p2 - 2*f2*f3*o1**2*o2*o3*p1*p3 + 2*f2*f3*o1**2*o2*o3*p2*p3 + 2*f2*f3*o1**2*o2*p1*p2 - 2*f2*f3*o1**2*o2*p1*p3 - 2*f2*f3*o1**2*o2*p2*p3 - 2*f2*f3*o1**2*o2*p3**2 - 2*f2*f3*o1**2*o3*p1*p2 + 2*f2*f3*

From AN EXPLICIT SOLUTION TO THE
PROBLEM OF OPTIMIZING THE
ALLOCATIONS OF A BETTOR’S WEALTH
WHEN WAGERING ON HORSE RACES - PETER SMOCZYNSKI, DAVE TOMKINS - 35_1_2.pdf

2.2) $E[r_{i}] = \frac{D \pi_{k}}{\beta_{k}} = \pi_{i}(Q_{i} + 1)$

where $D = 1 - tt$ and $tt$ is track take. 
$Q_{k}$ is the odds of horse k (i.e. a profit of $Q_{k}$ dollars for each $1 bet, if horse k wins). 
$\pi_{i}$ is probabilities for each horse.

6.8) $R(s) = 1 - \sum_{m \in S} f_{m} = \frac{D\sum_{k\notin S} \pi_{k}}{D - \sum_{k \in S}\beta_{k}}$, if $S \neq \emptyset$ and $R(\emptyset) = 1$

$\beta_{k}$ is public odds
$f_{m}$ is the betting fraction.

6.5) $f_{i}^{\text{opt}} = \pi_{i} - \beta_{i} \frac{\sum_{k \notin s} \pi_{k}}{D - \sum_{k \notin S} \beta_{k}}$ if $i \in S$, $f_{j}^{\text{opt}} = 0$ if $j \notin S$. 

6.9) $ln(G^{\text{opt}} = L(f^{\text{opt}}) = \sum_{k \in S}\pi_{k} ln(\frac{D \pi_{k}}{\beta_{k}}) + \sum_{k \notin S} \pi_{k} ln(R(S))$.


#### Algorithm

1. Calculate expected revenue rates $E[r_{i}]$ according to (2.2)
2. Reorder the horses so that the sequence $E[r_{i}]$ is nonincreasing.
3. Set $S = \emptyset$, $i = 1$, and $R = 1$
4. Repeat:
    * if $E[r_{i}] > R$ then insert the $i$ th horse into the set $S$ and recalculate $R$ according to (6.8).
    * else set $S^{\text{opt}} = S$ and stop repetition.

The set $S^{\text{opt}}$ produced by this algorithm is the optimal set of horses. When the optimal set of
horses is determined then the optimal allocation fractions should be determined from (6.5) and
the maximum of the average of the logarithm of the growth rate from (6.9).


In [236]:
def exp_ri(probability, odds):
    return [p*(o + 1) for p, o in zip(probability, odds)]
    
def exp_ri2(win_prob, belief_prob, track_take):
    return sorted([(w, b, (1 - track_take) * w / b) for w, b in zip(win_prob, belief_prob)], key=lambda x: x[2], reverse=True)
    
def reserve_rate(S, s_index, track_take):
    d = (1 - track_take)
    #print(enumerate(S))
    s_in = [e[1] for i,e in enumerate(S) if i in s_index]
    s_notin = [e[0] for i, e in enumerate(S) if i not in s_index]
    res = (d * sum(s_notin)) / (d - sum(s_in))
    return res

def f_opt(S, s_index, track_take):
    d = (1 - track_take)
    s_win = sum([e[0] for i, e in enumerate(S) if i not in s_index])
    s_belief = sum([e[1] for i, e in enumerate(S) if i in s_index])
    denominator = d - s_belief
    return [S[i][0] - (S[i][1] * s_win) / denominator for i in s_index]
    
    
    
win_prob = [0.003247, 0.003247, 0.003247, 0.01623, 0.2273, 0.1623, 0.5844]
belief_prob = [0.025, 0.2375, 0.0625, 0.125, 0.25, 0.3125, 0.1875]

win_prob = [0.25, 0.1, 0.1, 0.4, 0.15]
belief_prob = [0.17, 0.05667, 0.034, 0.34, 0.3993]

expected_revenue = exp_ri2(win_prob, belief_prob, .20)
display(expected_revenue)

setofs = []
i = 0
R = 1
while expected_revenue[i][2] > R:
    setofs.append(i)
    R = reserve_rate(expected_revenue, setofs, .2)
    i += 1
s_opt = [e for i, e in enumerate(expected_revenue) if i in setofs]

#    break
    #i += 1
#        
#        #setofs.append(expected_revenue[i])
#    #    R = pass 
#    #else:
 
#reserve_rate(expected_revenue, [1,3,4], .8)
print(s_opt)
print(f_opt(expected_revenue, setofs, .2))
    

[(0.1, 0.034, 2.3529411764705883),
 (0.1, 0.05667, 1.411681665784366),
 (0.25, 0.17, 1.1764705882352942),
 (0.4, 0.34, 0.9411764705882354),
 (0.15, 0.3993, 0.3005259203606311)]

[(0.1, 0.034, 2.3529411764705883), (0.1, 0.05667, 1.411681665784366), (0.25, 0.17, 1.1764705882352942), (0.4, 0.34, 0.9411764705882354)]
[0.07441428786434556, 0.05735463803742539, 0.12207143932172779, 0.1441428786434556]


In [262]:

test_frame = pd.DataFrame({"Win": [0.003247, 0.003247, 0.003247, 0.01623, 0.2273, 0.1623, 0.5844], "Belief": [0.025, 0.2375, 0.0625, 0.125, 0.25, 0.3125, 0.1875]})
display(test_frame.dtypes)

def exp_ri_df(dframe, track_take):
    D = 1 - track_take
    result_frame = dframe.copy()
    result_frame["Expected_revenue_rate"] = result_frame.apply(lambda x: (D * x["Win"]) / x["Belief"], axis=1)
    result_frame["Expected_revenue_rate2"] = [(1 - track_take) * w / b for w, b in zip(dframe.Win, dframe.Belief)]
    result_frame = result_frame.sort_values("Expected_revenue_rate", ascending=False)
    return result_frame
    
def reserve_rate_df(dframe, track_take):
    D = 1 - track_take
    return (D * dframe[dframe["S"] == False]["Win"].sum()) / (D - dframe[dframe["S"] == True]["Belief"].sum()) 

def f_opt_df(dframe, track_take):
    D = 1 - track_take
    c = dframe[dframe["S"] == False]["Win"].sum() / (D - dframe[dframe["S"] == True]["Belief"].sum())
    dframe.loc[:, "f_opt"] = dframe.apply(lambda x: x["Win"] - x["Belief"] * c if x["S"] == 1 else 0, axis=1)
    return dframe
def max_avg_log_growth_rate(dframe , track_take):
    D = 1 - track_take
    a = dframe[dframe["S"] == True].apply(lambda x: x["Win"] * np.log(D * x["Win"] / x["Belief"]), axis=1).sum()
    b = dframe[dframe["S"] == False].apply(lambda x: x["Win"] * np.log(reserve_rate_df(dframe, track_take)), axis=1).sum()
    return a + b
    
    
def calc_f_opt(df):
    #display(df)
    df = exp_ri_df(df, .2)
    #display(df)
    df = df.reset_index()
    df["S"] = 0
    df.loc[0, "S"] = 1
    df.reset_index()
    df["Reserve_rate"] = 0
    df.loc[0, "Reserve_rate"] =1
    R = 1
    for i in df.index:  
        if df.loc[i, "Expected_revenue_rate"] > R:
            df.loc[i, "S"] = 1
        df.loc[i, "Reserve_rate"] = R
        R = reserve_rate_df(df, .2)
    display( max_avg_log_growth_rate(df, .2))
    return f_opt_df(df, .2)
    #max_avg_log_growth_rate(df, .2)
    
calc_f_opt(test_frame)



#s = [1,3,6]

#[(i, e) for i,e in enumerate(range(2,10)) if i not in s]

Win       float64
Belief    float64
dtype: object

0.29623185361684173

,index,Win,Belief,Expected_revenue_rate,Expected_revenue_rate2,S,Reserve_rate,f_opt
0,6,0.584400,0.1875,2.493440,2.493440,1,1.000000,0.487018
1,4,0.227300,0.2500,0.727360,0.727360,1,0.542787,0.097458
2,5,0.162300,0.3125,0.415488,0.415488,0,0.415495,0.000000
3,0,0.003247,0.0250,0.103904,0.103904,0,0.415495,0.000000
4,3,0.016230,0.1250,0.103872,0.103872,0,0.415495,0.000000
5,2,0.003247,0.0625,0.041562,0.041562,0,0.415495,0.000000
6,1,0.003247,0.2375,0.010937,0.010937,0,0.415495,0.000000


In [263]:
test_df2 = pd.DataFrame({"Win": [0.25, 0.1, 0.1, 0.4, 0.15], "Belief": [0.17, 0.05667, 0.034, 0.34, 0.3993]})

calc_f_opt(test_df2)



0.060304161109983004

,index,Win,Belief,Expected_revenue_rate,Expected_revenue_rate2,S,Reserve_rate,f_opt
0,2,0.10,0.03400,2.352941,2.352941,1,1.000000,0.074414
1,1,0.10,0.05667,1.411682,1.411682,1,0.939948,0.057355
2,0,0.25,0.17000,1.176471,1.176471,1,0.902260,0.122071
3,3,0.40,0.34000,0.941176,0.941176,1,0.815827,0.144143
4,4,0.15,0.39930,0.300526,0.300526,0,0.602017,0.000000


In [283]:
precision_example1 = pd.DataFrame({"Adj_prob":[0.043578, 0.174312, 0.008716, 0.087156, 0.054472, 0.108945, 0.008716, 0.043578, 0.108945, 0.102536, 0.058104, 0.108945, 0.07263, 0.019368], "Line":[20, 5, 100, 10, 16, 8, 100, 20, 8, 8.5, 15, 8, 12, 45], "adj_odd": [22.9, 5.7, 114.7, 11.5, 18.4, 9.2, 114.7, 22.9, 9.2, 9.8, 17.2, 9.2, 13.8, 51.6],"Real_odd":[16, 7.2, 61, 12, 6.6, 5.8, 68, 15, 7.8, 8.5, 61, 5.4, 74, 18]  })

#display(1/precision_example1["adj_odd"])
#display(precision_example1["Adj_prob"])
#display(1/precision_example1["Real_odd"])
precision_example1["Real_prob"] = 1/(precision_example1["Real_odd"])
precision_example1["Real_prob"] = precision_example1["Real_prob"] / precision_example1["Real_prob"].sum()
#display(precision_example1['Real_prob'].sum())


precision_example1["Win"] = precision_example1['Adj_prob']
precision_example1["Belief"] = precision_example1["Real_prob"]
precision_example1 = calc_f_opt(precision_example1)

precision_example1["frac_01_bet"] = precision_example1["f_opt"] * .1 * 10000
precision_example1["frac_02_bet"] = precision_example1["f_opt"] * .2 * 10000
precision_example1["frac_03_bet"] = precision_example1["f_opt"] * .3 * 10000

display(precision_example1)

#(1/ precision_example1['Line']).sum()


0.09845365119413946

,index,Adj_prob,Line,adj_odd,Real_odd,Real_prob,Win,Belief,Expected_revenue_rate,Expected_revenue_rate2,S,Reserve_rate,f_opt,frac_01_bet,frac_02_bet,frac_03_bet
0,12,0.072630,12.0,13.8,74.0,0.012270,0.072630,0.012270,4.735625,4.735625,1,1.000000,0.060112,60.111797,120.223595,180.335392
1,10,0.058104,15.0,17.2,61.0,0.014842,0.058104,0.014842,3.131827,3.131827,1,0.941816,0.042961,42.961013,85.922026,128.883039
2,1,0.174312,5.0,5.7,7.2,0.112222,0.174312,0.112222,1.242628,1.242628,1,0.899760,0.059816,59.816244,119.632487,179.448731
3,3,0.087156,10.0,11.5,12.0,0.070786,0.087156,0.070786,0.985010,0.985010,1,0.841520,0.014936,14.935600,29.871200,44.806799
4,9,0.102536,8.5,9.8,8.5,0.096865,0.102536,0.096865,0.846837,0.846837,1,0.824301,0.003708,3.708084,7.416168,11.124252
5,8,0.108945,8.0,9.2,7.8,0.104570,0.108945,0.104570,0.833470,0.833470,1,0.819873,0.002256,2.255772,4.511545,6.767317
6,5,0.108945,8.0,9.2,5.8,0.135326,0.108945,0.135326,0.644045,0.644045,0,0.816212,0.000000,0.000000,0.000000,0.000000
7,0,0.043578,20.0,22.9,16.0,0.054130,0.043578,0.054130,0.644045,0.644045,0,0.816212,0.000000,0.000000,0.000000,0.000000
8,11,0.108945,8.0,9.2,5.4,0.143784,0.108945,0.143784,0.606160,0.606160,0,0.816212,0.000000,0.000000,0.000000,0.000000
9,7,0.043578,20.0,22.9,15.0,0.057514,0.043578,0.057514,0.606160,0.606160,0,0.816212,0.000000,0.000000,0.000000,0.000000


In [282]:
example2 = pd.DataFrame({"Win":[0.1758459, .3516918, .1099037, 0.087923, 0.1099037, 0.0439615, 0.039965, 0.0293077, 0.0251208, 0.0146536,0.0058615, 0.0058615, 0.000000008792, 0.000000008792], "Belief": [3, 4, 6, 7.5, 12, 13, 18, 20, 20, 50, 100, 120, 999, 999]})

example2["Belief"] = 1/ (example2["Belief"])
example2["Belief"] = example2["Belief"] / example2["Belief"].sum()

display(example2["Win"].sum())
display(example2["Belief"].sum())
example2 = calc_f_opt(example2)

example2["bet"] = example2["f_opt"] * 10000
example2["frac_01_bet"] = example2["f_opt"] * .1 * 10000
example2["frac_02_bet"] = example2["f_opt"] * .2 * 10000
example2["frac_03_bet"] = example2["f_opt"] * .3 * 10000



#display(example2["Belief"].sum())

display(example2)




0.999999717584

1.0

0.033788957345872214

,index,Win,Belief,Expected_revenue_rate,Expected_revenue_rate2,S,Reserve_rate,f_opt,bet,frac_01_bet,frac_02_bet,frac_03_bet
0,1,3.516918e-01,0.201697,1.394929,1.394929,1,1.000000,0.147209,1472.08983,147.208983,294.417966,441.626949
1,4,1.099037e-01,0.067232,1.307746,1.307746,1,0.866863,0.041743,417.42761,41.742761,83.485522,125.228283
2,9,1.465360e-02,0.016136,0.726514,0.726514,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000
3,6,3.996500e-02,0.044822,0.713316,0.713316,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000
4,11,5.861500e-03,0.006723,0.697461,0.697461,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000
5,3,8.792300e-02,0.107572,0.653873,0.653873,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000
6,2,1.099037e-01,0.134465,0.653873,0.653873,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000
7,7,2.930770e-02,0.040339,0.581221,0.581221,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000
8,10,5.861500e-03,0.008068,0.581217,0.581217,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000
9,5,4.396150e-02,0.062061,0.566690,0.566690,0,0.811048,0.000000,0.00000,0.000000,0.000000,0.000000


In [287]:
example3 = pd.DataFrame({"Win":[0.1043486, 0.2318858, 0.1897247, 0.1304358, 0.0695657, 0.1043465, 0.0496898, 0.0347829, 0.0695657, 0.0052174, 0.0052174, 0.0052174, 0.0, 0.0], "Belief": [3, 4, 6, 7.5, 12, 13, 18, 20, 20, 50, 100, 120, 999, 999]})

example3["Belief"] = 1 / example3["Belief"]
example3["Belief"] = example3["Belief"] / example3["Belief"].sum()
example3 = calc_f_opt(example3)

example3["bet"] = example3["f_opt"] * 10000
example3["frac_01_bet"] = example3["f_opt"] * .1 * 10000
example3["frac_02_bet"] = example3["f_opt"] * .2 * 10000
example3["frac_03_bet"] = example3["f_opt"] * .3 * 10000

display(example3.sort_values("index"))


0.012758089371411721

,index,Win,Belief,Expected_revenue_rate,Expected_revenue_rate2,S,Reserve_rate,f_opt,bet,frac_01_bet,frac_02_bet,frac_03_bet
10,0,0.104349,0.268930,0.310411,0.310411,0,0.858573,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,0.231886,0.201697,0.919737,0.919737,1,0.888439,0.015421,154.208126,15.420813,30.841625,46.262438
2,2,0.189725,0.134465,1.128768,1.128768,1,0.947346,0.045415,454.147084,45.414708,90.829417,136.244125
3,3,0.130436,0.107572,0.970036,0.970036,1,0.904026,0.014988,149.878067,14.987807,29.975613,44.963420
6,4,0.069566,0.067232,0.827763,0.827763,0,0.858573,0.000000,0.000000,0.000000,0.000000,0.000000
1,5,0.104346,0.062061,1.345089,1.345089,1,0.979840,0.037742,377.418885,37.741888,75.483777,113.225665
5,6,0.049690,0.044822,0.886889,0.886889,1,0.863573,0.001586,15.864695,1.586469,3.172939,4.759408
7,7,0.034783,0.040339,0.689804,0.689804,0,0.858573,0.000000,0.000000,0.000000,0.000000,0.000000
0,8,0.069566,0.040339,1.379605,1.379605,1,1.000000,0.026273,262.727025,26.272703,52.545405,78.818108
11,9,0.005217,0.016136,0.258675,0.258675,0,0.858573,0.000000,0.000000,0.000000,0.000000,0.000000


In [289]:
example4 = pd.DataFrame({"Win":[0.4187741, 0.1522615, 0.1675097, 0.0837548, 0.0418774, 0.0418774, 0.02393, 0.02393, 0.0335019, 0.0041877, 0.0041877, 0.0041877, 0, 0], "Belief": [3, 4, 6, 7.5, 12, 13, 18, 20, 20, 50, 100, 120, 999, 999]})

example4["Belief"] = 1 / example4["Belief"]
example4["Belief"] = example4["Belief"] / example4["Belief"].sum()
example4 = calc_f_opt(example4)

example4["bet"] = example4["f_opt"] * 10000
example4["frac_01_bet"] = example4["f_opt"] * .1 * 10000
example4["frac_02_bet"] = example4["f_opt"] * .2 * 10000
example4["frac_03_bet"] = example4["f_opt"] * .3 * 10000

display(example4.sort_values("index"))




0.01659022864815561

,index,Win,Belief,Expected_revenue_rate,Expected_revenue_rate2,S,Reserve_rate,f_opt,bet,frac_01_bet,frac_02_bet,frac_03_bet
0,0,0.418774,0.268930,1.245750,1.245750,1,1.000000,0.138255,1382.552922,138.255292,276.510584,414.765877
4,1,0.152261,0.201697,0.603921,0.603921,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,0.167510,0.134465,0.996600,0.996600,1,0.875524,0.027250,272.502961,27.250296,54.500592,81.750888
3,3,0.083755,0.107572,0.622875,0.622875,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000
6,4,0.041877,0.067232,0.498300,0.498300,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000
5,5,0.041877,0.062061,0.539825,0.539825,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000
9,6,0.023930,0.044822,0.427115,0.427115,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000
8,7,0.023930,0.040339,0.474572,0.474572,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000
2,8,0.033502,0.040339,0.664399,0.664399,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000
11,9,0.004188,0.016136,0.207623,0.207623,0,0.834474,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
datetime.datetime.now() < datetime.datetime.now() - datetime.timedelta(days=1)

False